# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [139]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [140]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [141]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [142]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [143]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [144]:
np.linalg.matrix_rank(a1)

6

In [145]:
l, u = diy_lu(a1)

print(l)
print(u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]
[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [146]:
def check_all_leading_minors_non_zero(a):
    """ Check if all leading minors of a matrix `a` are non zero

    Simply by calculating N determinants
    """
    N = a.shape[0]

    for i in range(N):
        if np.linalg.det(a[:i, :i]) == 0:
            return False
    return True

print(check_all_leading_minors_non_zero(a))
print(check_all_leading_minors_non_zero(a1))

True
False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [135]:
def lup_decompose(a):
    """Construct the LUP decomposition of the input matrix.
    
    """
    N = a.shape[0]

    s = np.arange(N) # swap vector
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        # chose pivot
        pivot_index = j + np.argmax(np.abs(u[j:, j]))
        # swapping
        s[[j, pivot_index]] = s[[pivot_index, j]]
        u[[j, pivot_index]] = u[[pivot_index, j]]
        L[[j, pivot_index]] = L[[pivot_index, j]]
        L[:, [j, pivot_index]] = L[:, [pivot_index, j]]

        gamma = u[j + 1:, j] / u[j, j]
        lam[j + 1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u, s

def lup_compose(L, U, s):
    """reconstruct original matrix A after LUP decomposition

    L - lower triangular matrix
    U - upper triangular matrix
    s - swap vector

    after multiplication inverse permutation is applied
    https://stackoverflow.com/a/55737198/12652871
    """
    return (L @ U)[np.argsort(p)]

In [136]:
# test on a1
L, U, p = lup_decompose(a1)
a1 - lup_compose(L, U, p)

array([[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -2.220e-16, -1.110e-16,
         2.776e-16],
       [ 0.000e+00,  0.000e+00, -1.110e-16,  1.665e-16,  2.776e-16,
         5.551e-17],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -3.331e-16,
         2.776e-17],
       [ 0.000e+00,  0.000e+00,  1.665e-16,  1.665e-16, -5.551e-17,
         0.000e+00]])

In [138]:
# test on a
L, U, p = lup_decompose(a)
a - lup_compose(L, U, p)

array([[ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  0.000e+00, -2.220e-16, -1.110e-16,
         2.776e-16],
       [ 0.000e+00,  0.000e+00, -1.110e-16,  1.665e-16,  2.776e-16,
         5.551e-17],
       [ 0.000e+00,  0.000e+00,  0.000e+00,  0.000e+00, -3.331e-16,
         0.000e+00],
       [ 0.000e+00,  0.000e+00,  1.665e-16,  1.665e-16, -5.551e-17,
         0.000e+00]])